# 【WIP】落書き認識Webアプリを作ろう

- Author: Arata Furukawa ([github](https://github.com/ornew), [facebook](https://www.facebook.com/old.r.new))
- Contributor: Hideya Kawahara

このノートブックは、セミナーの資料として作成されています。ノートブックは、自由な編集、実行が可能です。Markdown形式でドキュメントも書き込めるため、必要に応じてメモを追記するなど、工夫してご利用ください。

参加者の皆様には後日データを配布いたしますが、編集も含めて持ち帰りたい場合は、画面上部のツールバーから、【File】タブを選び、【Download as】を選ぶことでローカルマシン上に保存することが可能です。

このノートブックはご自由にご利用頂けますが、インターネット上への無断での転載だけはご遠慮くださいますようお願いします。

## モデルの概要

今回、認識する落書きは、以下の10クラス(種類)です。

1. りんご(apple)
2. ベッド(bed)
3. 猫(cat)
4. 犬(dog)
5. 目(eye)
6. 魚(fish)
7. 草(grass)
8. 手(hand)
9. アイスクリーム(ice cream)
10. ジャケット(jacket)

28x28ピクセルのグレースケール画像から、上記のいずれの落書きであるかを**確率的に**予測します。

![](./img/1.png)

### ディープラーニング

モデルは(ディープ)ニューラルネットワークで実装します。

ニューラルネットワークとは、生物のニューロン(神経細胞)のネットワークを数理モデルで模倣することで、特定の課題解決能力を機械的に学習する、機械学習アルゴリズムの一種です。深い層で構成されるニューラルネットワークの学習を行うことをディープラーニングといいます。

ディープラーニングにおけるモデルの学習は、以下の流れで行います。

0. モデルのパラメータを初期化する
1. 学習用データに対する予測を計算する
2. 教師ラベルと予測結果の誤差を計算する
3. 誤差を最小化するようにモデルのパラメータを更新する
4. **誤差が十分に小さくなるまで**1-3を繰り返す

![](./img/2.png)

## 実装の流れ

このノートブックでは、以下の手順で、ディープラーニングを用いた落書き(Doodle)認識を行うWebアプリを作成します。

1. ["the Quick, Draw!"データセット](https://quickdraw.withgoogle.com/data)を学習用データとして準備する
2. [TensorFlow](https://www.tensorflow.org/)で落書きを認識するディープニューラルネットワークのモデルを実装する
3. [Amazon SageMaker](https://aws.amazon.com/jp/sagemaker/)でモデルを学習する
4. [TensorFlow.js](https://js.tensorflow.org/)を使ったWebアプリに学習済みモデルを組み込む
5. [Amazon S3](https://aws.amazon.com/jp/s3/)でWebアプリを公開する

![](./img/3.png)

## 実装する

まず、作業に必要なモジュールを読み込みます。

In [ ]:
import six               # Python 2と3の互換性を保つためのライブラリです
import numpy as np       # 行列などの科学数値計算をするためのライブラリです

import matplotlib.pyplot as plt # グラフを描画するライブラリです 下の行を実行するとノートブック上で画像がインライン表示されます
%matplotlib inline

import tensorflow as tf

### ①学習用データを準備する

学習データは、Google社が[クリエイティブ・コモンズ ライセンス バージョン4.0](https://creativecommons.org/licenses/by/4.0/)で公開している["the Quick, Draw!"データセット](https://quickdraw.withgoogle.com/data)を利用します。

データをダウンロードして、`./raw_data`ディレクトリに保存します。

ちなみに、Jupyterノートブックでは、「`!`」を先頭につけると、シェルコマンドを実行できます(Pythonの機能ではありません)。出力をPythonで使ったり、Pythonの変数を引数に使ったりも出来るので便利です。ここでは`wget`コマンドを使ってファイルをダウンロードします。

In [ ]:
URL = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap'
LABELS = [
    'apple', 'bed', 'cat', 'dog', 'eye',
    'fish', 'grass', 'hand', 'ice cream', 'jacket',
]

In [ ]:
!mkdir -p ./data ./raw_data
for l in LABELS:
    url = '{}/{}.npy'.format(URL, l)
    !wget -NP raw_data "$url"

各ラベルのデータファイルがダウンロードできていることを確認します。

In [ ]:
!ls ./raw_data

ダウンロードしたデータを読み込みます。

In [ ]:
raw_data = {label: np.load('raw_data/{}.npy'.format(label)) for label in LABELS}

各データの数を確認してみましょう。

In [ ]:
for label, data in six.iteritems(raw_data):
    print('{:10}: {}'.format(label, len(data)))

ためしに「猫」の画像を表示してみます。

In [ ]:
plt.imshow(np.reshape(raw_data['cat'][0], [28, 28]), cmap='gray_r')
plt.show()

次に、学習用と評価用のデータセットを準備します。

1. ダウンロードしたデータセットのうち10万件を取り出す
    - 数にばらつきがあると、学習で用いられる頻度がクラスごとに変わってしまいます
2. それぞれ画像データと教師ラベルの組み合わせに変換する
    - 教師ラベルは、クラス名の文字列(例:apple)ではなく、それぞれクラスごとに番号を割り当てます
3. 学習用と評価用に7:3で分ける
    - 学習に使われていないデータで精度の評価を行いたいため、評価用のデータは別に保存します
4. ランダムにシャッフル

クラスごとに割り当てる教師ラベルの番号を確認します。

In [ ]:
for i, label_name in enumerate(LABELS):
    print(u'番号: {}   ラベル名: {}'.format(i, label_name))

In [ ]:
train_data = []
test_data = []
for label_name, value in six.iteritems(raw_data):
    label_index = LABELS.index(label_name)
    print('proccessing label class {}: "{}"'.format(label_index, label_name))
    value = np.asarray(value) / 255.
    tr = value[:70000]
    te = value[70000:100000]
    train_data.extend(zip(tr, np.full(70000, label_index)))
    test_data.extend(zip(te, np.full(30000, label_index)))
np.random.shuffle(train_data)
np.random.shuffle(test_data)

学習用と評価用のデータを、TFRecord形式のファイルに出力します。TFRecordは非同期のストリーミング読み込みが可能で、圧縮効率が高く、機械学習で用いられる大規模データセットの保存に向いています。

In [ ]:
train_filename = './data/train.tfr'
test_filename  = './data/test.tfr'

def get_example_proto(image, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'image' : tf.train.Feature(float_list=tf.train.FloatList(value=image)),
        'label' : tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
    })).SerializeToString()

以下の変換処理は5分ほどかかります。少々お待ちください。

In [ ]:
%%time
tfr_options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
with tf.python_io.TFRecordWriter(train_filename, tfr_options) as train_tfr, \
     tf.python_io.TFRecordWriter(test_filename, tfr_options) as test_tfr:
    print('Converting train data...')
    for data, label in train_data:
        train_tfr.write(get_example_proto(data, [label]))
    print('Converting test data...')
    for data, label in test_data:
        test_tfr.write(get_example_proto(data, [label]))

`train.tfr`と`test.tfr`が生成されていれば成功です。

In [ ]:
!ls data

データの準備が完了しました。生成したデータは後ほどS3にアップロードします。

### ②TensorFlowでモデルの定義プログラムを実装する

モデルの実装には、TensorFlowを利用します。TensorFlowは、Google社を主体として開発されている、オープンソースの汎用的な分散数値演算ライブラリです。TensorFlowにはディープラーニング向けのライブラリが用意されており、現在世界で最も人気のディープラーニングフレームワークと言われています。

以下の4つの関数を定義したプログラムを用意するだけで、Amazon SageMakerを使ってモデルの学習を行うことができます。

```python
def train_input_fn(training_dir, hyperparameters):
    # 学習用の入力データを読み込みます

def eval_input_fn(training_dir, hyperparameters):
    # 評価用の入力データを読み込みます

def serving_input_fn(hyperparameters):
    # サービング(APIサーバにしたとき)用に受け取る入力データの形式を定義します

def model_fn(features, labels, mode, hyperparameters):
    # モデルの定義をします
```

もう少し具体的には、`model_fn`の中で以下の3つを定義します。

1. **モデル**: ニューラルネットワーク
2. **誤差**: 教師データと予測結果がどの程度違ったのかを定式化する
3. **最適化アルゴリズム**: 誤差を最小化するようにモデルを最適化するアルゴリズム

つまり、入力データと、上記3つの定義を行うだけです。

予め実装してあるので、簡単な解説を行います。

#### `train_input_fn`、`eval_input_fn`の実装

`train_input_fn`、`eval_input_fn`の引数には、S3上の学習データがマウントされたディレクトリのパスと、ジョブの起動時に指定できるプログラムの挙動を制御するためのハイパーパラメータが渡されます。

```python
def train_input_fn(training_dir, params):
    return _input_fn(training_dir, params, is_training=True)

def eval_input_fn(training_dir, params):
    return _input_fn(training_dir, params, is_training=False)

def _input_fn(data_dir, params, is_training):
    # ハイパーパラメータを取得します
    batch_size  = params.get('batch_size', 96)
    buffer_size = params.get('shuffle_buffer_size', 4096)
    cmp_type    = params.get('tfrecord_compression_type', 'GZIP')
    train_file  = params.get('train_tfrecord_file', 'train.tfr')
    test_file   = params.get('test_tfrecord_file', 'test.tfr')

    # 学習用データのパス
    tfrecord = os.path.join(data_dir,
        train_file if is_training else test_file)

    # TFrecordの読み込みと、シャッフルやリピートなどを設定します
    return (tf.data.TFRecordDataset(tfrecord, compression_type=cmp_type)
        .map(_parse_example)
        .shuffle(buffer_size)
        .batch(batch_size)
        .repeat(-1 if is_training else 1)
        .make_one_shot_iterator()
        .get_next())

# 圧縮のためtf.train.Exampleに変換されているのでパースする関数です
def _parse_example(example):
    features = tf.parse_single_example(example,  {
        'image': tf.FixedLenFeature([28, 28, 1], tf.float32),
        'label': tf.FixedLenFeature([]         , tf.int64),
    })
    label = features.pop('label')
    return features, label
```

#### `serving_input_fn`の実装

サービング（学習後の認識機能の提供）用の入力定義を行います。これはAPIサーバにデプロイしない場合も必要なので注意してください。

以下では、入力データは、キーを`image`とし、浮動小数型のテンソルであることを定義しています。`[None, 28, 28, 1]`は、データの形状を示しています。

TensorFlowで扱われるデータは全てTensor(テンソル)です。この場合、`image`テンソルはランク4のテンソルで、プログラム上では4次元配列で表現されます。各次元の長さは、None(可変長)、28、28、1です。

```python
def serving_input_fn(params):
    return tf.estimator.export.build_raw_serving_input_receiver_fn({
        'image': tf.placeholder(tf.float32, [None, 28, 28, 1], name='image')
    })()
```

#### `model_fn`の実装

`model_fn`では、モデルと誤差と最適化の定義が必要でした。該当部分は以下のようになっています(一部省略しています)。

```python

def model_fn(features, labels, mode, params):
    # ...

    # 第一引数のfeaturesが入力データです
    image = features['image']

    # ...

    #=========================================================
    # ニューラルネットワークを定義します
    #=========================================================
    with tf.variable_scope('model', initializer=initializer):
        x = image
        x = tf.layers.conv2d(x, 32, 5, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, 2, 2, padding='SAME')
        x = tf.layers.conv2d(x, 64, 5, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, 2, 2, padding='SAME')
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, 1024, activation=tf.nn.relu)
        x = tf.layers.dropout(x, rate=dropout_rate, training=is_training)
        x = tf.layers.dense(x, 10)
        logits = x

    # 予測結果: クラスごとの離散確率分布、最も確率の高いクラスのインデクス
    predictions = {
        'probabilities': tf.nn.softmax(logits),
        'classes'      : tf.argmax(logits, axis=1),
    }
    
    # ...

    #=========================================================
    # モデルの誤差を定義します
    #=========================================================
    with tf.variable_scope('losses'):
        # クロスエントロピーを計算して誤差に追加します
        cross_entropy_loss = tf.losses.sparse_softmax_cross_entropy(
            labels=labels, logits=logits)
        
        # モデルで追加された全ての誤差の総和を取得します
        total_loss = tf.losses.get_total_loss()

    # ...

    #=========================================================
    # モデルを学習(=パラメータを最適化)します
    #=========================================================
    global_step = tf.train.get_or_create_global_step()
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.variable_scope('optimizer'), tf.control_dependencies(update_ops):
        # total_loss(誤差の総和)が小さくなるようにパラメータを更新します
        optimizer = tf.train.AdamOptimizer(learning_rate)
        fit = optimizer.minimize(total_loss, global_step)

    # ...
```

モデルの定義に関する詳細は、別途ノートブック`model.ipynb`で解説しています。

### ③Amazon SageMakerでモデルを学習する

Amazon SageMaker SDKを使い、プログラムとデータを指定して学習を実行します。

まずは必要な情報を取得します。

In [ ]:
import sagemaker

role    = sagemaker.get_execution_role()
session = sagemaker.Session()
bucket  = session.default_bucket()

次に、データの保存先などの文字列を変数で定義します。ハンズオンでは共用のストレージを利用するため、**各自で保存先など変えて頂きます**。

```python
your_name = 'arata-furukawa' # 例
```

上記のように、`your_name`にご自身の名前など**他の人と被らない文字列**を入れてから、セルを実行してください。

In [ ]:
your_name = '' # 半角英数字とハイフンのみ利用可能です

# 文字列チェック
import re; assert re.match(r'^[0-9a-z-]+$', your_name) is not None

def _s3(path):
    return 's3://{}/doodle/model/{}/{}'.format(bucket, your_name, path)
data_key_prefix = 'doodle/model/{}/data'.format(your_name)

config = dict(
    data_dir        = _s3('data'),
    output_path     = _s3('export'),
    checkpoint_path = _s3('ckpt'),
    code_location   = _s3('src'),
    public_dir      = _s3('public'),
    job_name        = 'doodle-training-job-{}'.format(your_name)
)

In [ ]:
for k, v in six.iteritems(config):
    print('key: {:20}, value: {:20}'.format(k, v))

上記で設定したユニークなS3パスに、①で作成したデータセットをアップロードします。

In [ ]:
uploaded_data_dir = session.upload_data(
    'data', # ローカルディレクトリ
    bucket=bucket,
    key_prefix=data_key_prefix)
assert uploaded_data_dir == config['data_dir']

モデルの学習では、「エスティメータ(Estimator)」を利用します。 エスティメータは、モデルの学習や評価、保存やデプロイなどを行う抽象化されたインターフェイスです。

用意したパスなどを設定として渡して、エスティメータを作成します。

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    # ハイパーパラメータ
    # ②で定義したプログラムの引数に渡されます
    hyperparameters={
        'save_summary_steps': 100,
    },
    
    # 先程設定した、各データの保存先のパス
    output_path     = config['output_path'],
    checkpoint_path = config['checkpoint_path'],
    code_location   = config['code_location'],
    
    # 学習用プログラムに関する設定
    source_dir='./src',      # 学習用のプログラムが保存されたローカルディレクトリ
    entry_point='doodle.py', # ②で定義した学習用プログラムのファイル名
    framework_version='1.6', # 利用したいTensorFlowのバージョン
    
    training_steps=20000,
    evaluation_steps=2000,
    
    # AWSでの実行に関する設定
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge')

エスティメータに対して、学習用データのパス名を指定して`fit`関数を呼び出すと、学習ジョブが作成され、クラウド上でモデルの学習が実行されます。

この学習には10〜15分かかります。実行完了までしばらくお待ち下さい。

In [ ]:
%%time
estimator.fit(config['data_dir'], job_name=config['job_name'],
              wait=True, run_tensorboard_locally=True)

ちなみに、`run_tensorboard_locally`引数に`True`を渡すと、ノートブック上でTensorBoardが実行されます。TensorBoardに学習ログなどを表示するようにしてあるので、下記URLにアクセスして確認してみましょう。

`https://(ノートブックのURL)/`[proxy/6006/](/proxy/6006/)

### ④学習したモデルをダウンロードして、Webアプリに組み込む

- S3からモデルデータをダウンロード・解凍
- tfjs_converterでWeb向けに変換(ジョブで直接出力にするかも。ただしどのみちtar出力になっちゃうのでダウンロードして回答する必要あるしいらないかも)
- 事前に用意したWebアプリをダウンロード・解凍
- 適切なディレクトリに保存
- Webアプリの設定ファイルを編集

学習したモデルは、エスティメータの`output_path`引数で指定した場所にGZIP圧縮されたTar書庫で保存されています。

モデルをダウンロードして展開します。

In [ ]:
model_url = '{}/{}/output/model.tar.gz'.format(config['output_path'], config['job_name'])

In [ ]:
!rm -rf ./export ./model.tar.gz
!aws s3 cp "$model_url" ./model.tar.gz
!tar xvzf ./model.tar.gz

学習ジョブで生成されたモデルは、SavedModelという形式です。TensorFlow.jsでブラウザ上で実行するために、Web用のフォーマットに変換します。

In [ ]:
!saved_model_cli show --dir ./export/Servo/* --all

In [ ]:
!pip install tensorflowjs
!mkdir -p ./webapp/saved_model_js

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='probabilities,classes' \
    --saved_model_tags=serve \
    ./export/Servo/* \
    ./webapp/model

次に、Webアプリをダウンロードします。

In [ ]:
!mkdir -p ./webapp
!wget -O webapp.tar.gz https://github.com/maru-labo/doodle/releases/download/v0.0.2/tensorflowjs.doodle.marulabo.tar.gz
!tar xvzf webapp.tar.gz -C webapp

### ⑤WebアプリをS3でホスティングして公開する

- S3上にアップロード
- アクセスして確認する

In [ ]:
!aws s3 sync ./webapp $public_dir

In [ ]:
print('https://s3-{}.amazonaws.com/{}/index.html'.format(session.boto_region_name, public_dir[5:]))